In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
import warnings
warnings.filterwarnings('ignore')
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
file_path = '/content/drive/MyDrive/Colab Notebooks/Rotman Datathon/country_means_dataset_final.csv'
uncleaned_data = pd.read_csv(file_path)

In [ ]:
uncleaned_data.head()

,Country Name,Country Code,Time,Time Code,Adjusted net national income (current US$) [NY.ADJ.NNTY.CD],Adjusted net national income per capita (current US$) [NY.ADJ.NNTY.PC.CD],"Air transport, freight (million ton-km) [IS.AIR.GOOD.MT.K1]","Air transport, registered carrier departures worldwide [IS.AIR.DPRT]",Broad money (% of GDP) [FM.LBL.BMNY.GD.ZS],Broad money growth (annual %) [FM.LBL.BMNY.ZG],...,Income share held by highest 20% [SI.DST.05TH.20],Income share held by lowest 10% [SI.DST.FRST.10],Income share held by lowest 20% [SI.DST.FRST.20],Income share held by second 20% [SI.DST.02ND.20],Income share held by third 20% [SI.DST.03RD.20],Adjusted savings: education expenditure (% of GNI) [NY.ADJ.AEDU.GN.ZS],Adjusted savings: education expenditure (current US$) [NY.ADJ.AEDU.CD],Human Development Index (HDI),Cost of Living,Region
0,Afghanistan,AFG,2016,1.769986e+09,1.680928e+10,5.028319e+02,5.150451e+01,1.732395e+04,3.516067e+01,1.210993e+01,...,1.769986e+09,1.769986e+09,1.769986e+09,1.769986e+09,1.769986e+09,2.711187,4.961485e+08,0.462000,354,South Asia
1,Albania,ALB,2016,1.016551e+09,1.120919e+10,3.897635e+03,2.185714e-02,4.905866e+03,8.130776e+01,5.071977e+00,...,3.962500e+01,3.250000e+00,7.912500e+00,1.248750e+01,1.686250e+01,3.140582,4.208859e+08,0.789000,608,Europe & Central Asia
2,Algeria,DZA,2016,1.416222e+10,1.371742e+11,3.445510e+03,1.843785e+01,5.715223e+04,7.051843e+01,8.856861e+00,...,3.720000e+01,4.000000e+00,9.400000e+00,1.370000e+01,1.750000e+01,4.467196,7.706673e+09,0.745000,431,Middle East & North Africa
3,American Samoa,ASM,2016,9.434172e+07,9.434172e+07,9.434172e+07,1.033649e-02,5.737038e+03,9.434172e+07,9.434172e+07,...,9.434172e+07,9.434172e+07,9.434172e+07,9.434172e+07,9.434172e+07,11.760709,9.434172e+07,0.727361,431,East Asia & Pacific
4,Andorra,AND,2016,4.040298e+08,4.040298e+08,4.040298e+08,4.040298e+08,4.040298e+08,4.040298e+08,4.040298e+08,...,4.040298e+08,4.040298e+08,4.040298e+08,4.040298e+08,4.040298e+08,2.789328,9.865100e+07,0.884000,1232,Europe & Central Asia


In [ ]:
new_data = uncleaned_data.drop(columns=['Country Name', 'Time', 'Time Code', 'Country Code', 'Region'])
new_data = new_data.drop(index=3)
new_data = new_data.replace("..", np.nan).apply(pd.to_numeric, errors='coerce')
new_data.iloc[:, 1:] = new_data.iloc[:, 1:].fillna(new_data.iloc[:, 1:].mean())

In [ ]:
new_data.columns[:]

Index(['Adjusted net national income (current US$) [NY.ADJ.NNTY.CD]',
       'Adjusted net national income per capita (current US$) [NY.ADJ.NNTY.PC.CD]',
       'Air transport, freight (million ton-km) [IS.AIR.GOOD.MT.K1]',
       'Air transport, registered carrier departures worldwide [IS.AIR.DPRT]',
       'Broad money (% of GDP) [FM.LBL.BMNY.GD.ZS]',
       'Broad money growth (annual %) [FM.LBL.BMNY.ZG]',
       'Bank capital to assets ratio (%) [FB.BNK.CAPA.ZS]',
       'Bank liquid reserves to bank assets ratio (%) [FD.RES.LIQU.AS.ZS]',
       'Bank nonperforming loans to total gross loans (%) [FB.AST.NPER.ZS]',
       'Central government debt, total (% of GDP) [GC.DOD.TOTL.GD.ZS]',
       ...
       'Income share held by highest 10% [SI.DST.10TH.10]',
       'Income share held by highest 20% [SI.DST.05TH.20]',
       'Income share held by lowest 10% [SI.DST.FRST.10]',
       'Income share held by lowest 20% [SI.DST.FRST.20]',
       'Income share held by second 20% [SI.DST.02ND.

In [ ]:
def top_y_correlated_features(dataset, top_x, y_header):
  y_col = dataset.columns.get_loc(y_header)
  corr_matrix = dataset.corr().iloc[:, y_col:y_col+1].transform(abs)
  return corr_matrix.sort_values(by=[y_header], ascending=False).index.values[:top_x]

In [ ]:
important_cols = ['Consumer price index (2010 = 100) [FP.CPI.TOTL]',
                  "Adjusted net national income per capita (current US$) [NY.ADJ.NNTY.PC.CD]",
                  'Households and NPISHs final consumption expenditure (% of GDP) [NE.CON.PRVT.ZS]',
                  'Households and NPISHs Final consumption expenditure (current US$) [NE.CON.PRVT.CD]',
                  'GDP per capita (current US$) [NY.GDP.PCAP.CD]',
                  'Inflation, GDP deflator (annual %) [NY.GDP.DEFL.KD.ZG]']

In [ ]:
def correlation(dataset, threshold, top_x, y_header):
  col_corr = set()
  corr_matrix = dataset.corr()
  for i in range(len(corr_matrix.columns)):
    for j in range(i):
      if abs(corr_matrix.iloc[i, j]) > threshold:
        colname = corr_matrix.columns[i]
        if colname not in top_y_correlated_features(new_data, top_x, y_header) and colname not in important_cols:
          col_corr.add(colname)
  return col_corr

In [ ]:
new_data.head()

,Adjusted net national income (current US$) [NY.ADJ.NNTY.CD],Adjusted net national income per capita (current US$) [NY.ADJ.NNTY.PC.CD],"Air transport, freight (million ton-km) [IS.AIR.GOOD.MT.K1]","Air transport, registered carrier departures worldwide [IS.AIR.DPRT]",Broad money (% of GDP) [FM.LBL.BMNY.GD.ZS],Broad money growth (annual %) [FM.LBL.BMNY.ZG],Bank capital to assets ratio (%) [FB.BNK.CAPA.ZS],Bank liquid reserves to bank assets ratio (%) [FD.RES.LIQU.AS.ZS],Bank nonperforming loans to total gross loans (%) [FB.AST.NPER.ZS],"Central government debt, total (% of GDP) [GC.DOD.TOTL.GD.ZS]",...,Income share held by highest 10% [SI.DST.10TH.10],Income share held by highest 20% [SI.DST.05TH.20],Income share held by lowest 10% [SI.DST.FRST.10],Income share held by lowest 20% [SI.DST.FRST.20],Income share held by second 20% [SI.DST.02ND.20],Income share held by third 20% [SI.DST.03RD.20],Adjusted savings: education expenditure (% of GNI) [NY.ADJ.AEDU.GN.ZS],Adjusted savings: education expenditure (current US$) [NY.ADJ.AEDU.CD],Human Development Index (HDI),Cost of Living
0,1.680928e+10,5.028319e+02,5.150451e+01,1.732395e+04,3.516067e+01,1.210993e+01,1.769986e+09,6.812781e+01,1.293410e+01,1.769986e+09,...,1.769986e+09,1.769986e+09,1.769986e+09,1.769986e+09,1.769986e+09,1.769986e+09,2.711187,4.961485e+08,0.462,354
1,1.120919e+10,3.897635e+03,2.185714e-02,4.905866e+03,8.130776e+01,5.071977e+00,8.523997e+00,1.656783e+01,1.369670e+01,7.431936e+01,...,2.398750e+01,3.962500e+01,3.250000e+00,7.912500e+00,1.248750e+01,1.686250e+01,3.140582,4.208859e+08,0.789,608
2,1.371742e+11,3.445510e+03,1.843785e+01,5.715223e+04,7.051843e+01,8.856861e+00,8.714937e+00,4.324201e+01,1.461504e+01,1.416222e+10,...,2.290000e+01,3.720000e+01,4.000000e+00,9.400000e+00,1.370000e+01,1.750000e+01,4.467196,7.706673e+09,0.745,431
4,4.040298e+08,4.040298e+08,4.040298e+08,4.040298e+08,4.040298e+08,4.040298e+08,4.040298e+08,4.040298e+08,4.040298e+08,4.040298e+08,...,4.040298e+08,4.040298e+08,4.040298e+08,4.040298e+08,4.040298e+08,4.040298e+08,2.789328,9.865100e+07,0.884,1232
5,5.479143e+10,1.999320e+03,5.642863e+01,1.162892e+04,3.369652e+01,1.512578e+01,1.025546e+01,4.046554e+01,1.241874e+01,6.206190e+09,...,3.960000e+01,5.560000e+01,1.300000e+00,3.800000e+00,7.700000e+00,1.260000e+01,3.550192,2.858503e+09,0.591,896


In [ ]:
y_vars = ['Cost to export, border compliance (US$) [IC.EXP.CSBC.CD]',
          'Cost to import, border compliance (US$) [IC.IMP.CSBC.CD]',
          'Exports of goods and services (annual % growth) [NE.EXP.GNFS.KD.ZG]',
          'Logistics performance index: Overall (1=low to 5=high) [LP.LPI.OVRL.XQ]',
          'Transport services (% of service imports, BoP) [BM.GSR.TRAN.ZS]',
          'Air transport, freight (million ton-km) [IS.AIR.GOOD.MT.K1]',
          'Trade (% of GDP) [NE.TRD.GNFS.ZS]',
          'Energy imports, net (% of energy use) [EG.IMP.CONS.ZS]',
          'Broad money (% of GDP) [FM.LBL.BMNY.GD.ZS]'
          ]

In [ ]:
def feature_selection(y_var):
  y = new_data[y_var]
  x = new_data.drop(columns=y_vars)
  x = x.rename(columns=lambda col: col.replace('[', '_').replace(']', '_').replace('<', '_'))
  X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)
  corr_features = correlation(X_train, 0.85, 20, y_var)
  X_test = X_test.drop(corr_features, axis=1)
  X_train = X_train.drop(corr_features, axis=1)
  return X_train, X_test, y_train, y_test

In [ ]:
models = [
    ("Linear Regression", LinearRegression()),
    ("Ridge", Ridge()),
    ("Random Forest", RandomForestRegressor(random_state=0, n_jobs=-1, n_estimators=5)),
    ("XGB Regressor", XGBRegressor(random_state=0, n_jobs=-1)),
    ("Decison Tree", DecisionTreeRegressor(random_state=0,
                           max_depth=5,
                           min_samples_split=10,
                           min_samples_leaf=1)),
    ("Gradient Boosting", GradientBoostingRegressor(random_state=0)),
    ("AdaBoost", AdaBoostRegressor(random_state=0)),
    ("KNN", KNeighborsRegressor())
]

In [ ]:
def evaluate_model(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    mape_score = mean_absolute_percentage_error(y_test, y_pred)
    return {"MSE": mse,"R2 Score": r2, "MAPE": mape_score}

In [ ]:
X_train, X_test, y_train, y_test = feature_selection(y_vars[0])

In [ ]:
def run_models():
  for y_var in y_vars:
    X_train, X_test, y_train, y_test = feature_selection(y_var)
    # print(y_var)
    results = list()
    for name, model in models:
      eval_metrics = evaluate_model(model, X_train, X_test, y_train, y_test)
      results.append([name, eval_metrics["MSE"], eval_metrics["R2 Score"], eval_metrics["MAPE"]])
      # print(name, results["MSE"], results["R2 Score"], results["MAPE"])
    results.sort(key=lambda x: (x[1], x[3], x[2]), reverse=False)
    df = pd.DataFrame(results, columns=["Model", "MSE", "R2 Score", "MAPE"])
    df = df.style.set_caption(y_var)
    print()
    display(df)

In [ ]:
run_models()

,Model,MSE,R2 Score,MAPE
0,Decison Tree,145295781390153056.000000,-49.252130,120868826991075856.000000
1,XGB Regressor,104135199859488320.000000,-35.016294,266593781670078840832.000000
2,Ridge,100993780686925856.000000,-33.929800,45459530836436511096832.000000
3,Linear Regression,100469831259622816.000000,-33.748587,45928549823177419653120.000000
4,Gradient Boosting,29580377469001380.000000,-9.230696,3387920846509533184.000000
5,Random Forest,23847779244922876.000000,-7.248014,16584278196049082.000000
6,KNN,9897574989037754.000000,-2.423184,76517149826389792.000000
7,AdaBoost,868941725411633.875000,0.699467,5594142677601177567232.000000


,Model,MSE,R2 Score,MAPE
0,Decison Tree,145423672238834400.000000,-49.296399,274990421046449664.000000
1,XGB Regressor,103534464389269664.000000,-34.808549,38012034420539621376.000000
2,Ridge,100993457935080368.000000,-33.929714,70170409852217078054912.000000
3,Linear Regression,100469491652806352.000000,-33.748494,70966969795248486612992.000000
4,Gradient Boosting,29500323386340044.000000,-9.203016,7564509806767003648.000000
5,Random Forest,23675101683793712.000000,-7.188298,165981985086446208.000000
6,KNN,9897568405739966.000000,-2.423184,175094972600622720.000000
7,AdaBoost,984251253134523.250000,0.659586,11188326863765961179136.000000


,Model,MSE,R2 Score,MAPE
0,Linear Regression,2886383494374992379904.000000,-298188.358954,8103679567.352539
1,Ridge,2877975376961306361856.000000,-297319.724849,8116115697.973225
2,KNN,1336023983170606336.000000,-137.023286,402824362.244968
3,Random Forest,436472825136040192.000000,-44.091566,60644523.776599
4,XGB Regressor,27179843708771660.000000,-1.807922,19343914.140311
5,Gradient Boosting,19953588035964300.000000,-1.061385,26343989.019532
6,AdaBoost,9588240046657122.000000,0.009449,77742719.157657
7,Decison Tree,4529455018223964.000000,0.532067,7283473.058784


,Model,MSE,R2 Score,MAPE
0,Random Forest,15004095270174380.000000,-2.159209,11457256.806431
1,Ridge,9601375322142814.000000,-1.021631,16578447.008588
2,Linear Regression,9332659867060402.000000,-0.965052,16677151.591855
3,KNN,6612566357592996.000000,-0.392318,8629405.343460
4,Gradient Boosting,5942401861210666.000000,-0.251211,9690452.806881
5,AdaBoost,5914740877687463.000000,-0.245387,19342046.571336
6,XGB Regressor,3194199801850154.000000,0.327441,6119061.095914
7,Decison Tree,2652706165394847.000000,0.441456,1785553.688736


,Model,MSE,R2 Score,MAPE
0,KNN,15253660224387833856.000000,0.261220,5147368.464750
1,Random Forest,1511087137629611008.000000,0.926814,0.367929
2,AdaBoost,543431112388158912.000000,0.973680,637027.364521
3,XGB Regressor,540253238934331392.000000,0.973834,41915.699625
4,Gradient Boosting,431519859615949888.000000,0.979100,7302.843828
5,Decison Tree,417658562980776064.000000,0.979772,162697.844871
6,Linear Regression,1643.798866,1.000000,0.665581
7,Ridge,1491.333184,1.000000,0.657833


,Model,MSE,R2 Score,MAPE
0,KNN,10233636845234491392.000000,0.265863,13459260892512264388608.000000
1,Random Forest,4710142265414949888.000000,0.662106,20606276866016184107008.000000
2,Gradient Boosting,269560574876152928.000000,0.980662,10581857967798505963520.000000
3,Decison Tree,263669757970504512.000000,0.981085,42732546735689049309184.000000
4,AdaBoost,263547811112319680.000000,0.981094,18574163262166626467840.000000
5,XGB Regressor,261659410410988448.000000,0.981229,30480654128386779643904.000000
6,Linear Regression,295820563557718.625000,0.999979,7116227046820053778432.000000
7,Ridge,264599231867857.406250,0.999981,6261773819757965869056.000000


,Model,MSE,R2 Score,MAPE
0,KNN,1325446344481700096.000000,-138.036233,5269885.323250
1,Decison Tree,5134093603221732.000000,0.461446,1863.124721
2,AdaBoost,3081723197885696.000000,0.676734,166520.937495
3,XGB Regressor,1125750396582599.500000,0.881911,431.638562
4,Random Forest,994500116027222.500000,0.895679,1411.076543
5,Gradient Boosting,126446917991486.906250,0.986736,278.119010
6,Ridge,1420.253608,1.000000,0.271945
7,Linear Regression,1416.426093,1.000000,0.271909


,Model,MSE,R2 Score,MAPE
0,KNN,64095944509248760.000000,0.010112,2336447.288685
1,Decison Tree,6031463553478140.000000,0.906851,20692.450399
2,AdaBoost,4377474971349375.500000,0.932395,747942.639072
3,Random Forest,1064575558128716.500000,0.983559,4142.150746
4,XGB Regressor,315764037657207.562500,0.995123,4012.448778
5,Gradient Boosting,58995873144121.031250,0.999089,2905.226737
6,Linear Regression,165987113.942530,1.000000,56.660182
7,Ridge,150222992.929655,1.000000,62.927811


,Model,MSE,R2 Score,MAPE
0,KNN,463276786209320599552.000000,-2.841619,28343802.820442
1,Decison Tree,102571328161677295616.000000,0.149450,79699.977883
2,Gradient Boosting,40600150393036234752.000000,0.663332,14708.112094
3,AdaBoost,20427513665533218816.000000,0.830609,10235099.509635
4,XGB Regressor,18742046684130525184.000000,0.844586,56065.746335
5,Random Forest,13290074229895147520.000000,0.889795,0.548231
6,Linear Regression,6669.638120,1.000000,0.544470
7,Ridge,6509.208335,1.000000,0.557341
